#TWEETS DISASTER

Spark session

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

install necessaire sur le cluster

In [0]:
%pip install spark-nlp


Python interpreter will be restarted.
Python interpreter will be restarted.


Import necessaire

chargement du dataframe

In [0]:
dbutils.fs.ls("/FileStore/tables")


Out[1]: [FileInfo(path='dbfs:/FileStore/tables/train-1.csv', name='train-1.csv', size=987712, modificationTime=1730380438000),
 FileInfo(path='dbfs:/FileStore/tables/train.csv', name='train.csv', size=987712, modificationTime=1730365543000)]

In [0]:
from pyspark.sql.functions import lit

df_train = spark.read.format("csv").option("header", True) \
                     .load("/FileStore/tables/train.csv")

df_train.show(20)


+---+-------+--------+--------------------+------+
| id|keyword|location|                text|target|
+---+-------+--------+--------------------+------+
|  1|   null|    null|Our Deeds are the...|     1|
|  4|   null|    null|Forest fire near ...|     1|
|  5|   null|    null|All residents ask...|     1|
|  6|   null|    null|13,000 people rec...|     1|
|  7|   null|    null|Just got sent thi...|     1|
|  8|   null|    null|#RockyFire Update...|     1|
| 10|   null|    null|#flood #disaster ...|     1|
| 13|   null|    null|I'm on top of the...|     1|
| 14|   null|    null|There's an emerge...|     1|
| 15|   null|    null|I'm afraid that t...|     1|
| 16|   null|    null|Three people died...|     1|
| 17|   null|    null|Haha South Tampa ...|     1|
| 18|   null|    null|#raining #floodin...|     1|
| 19|   null|    null|#Flood in Bago My...|     1|
| 20|   null|    null|Damage to school ...|     1|
| 23|   null|    null|      What's up man?|     0|
| 24|   null|    null|       I 

In [0]:
%pip freeze

argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
attrs==21.4.0
backcall==0.2.0
backports.entry-points-selectable==1.2.0
beautifulsoup4==4.11.1
black==22.3.0
bleach==4.1.0
boto3==1.21.32
botocore==1.24.32
certifi==2021.10.8
cffi==1.15.0
chardet==4.0.0
charset-normalizer==2.0.4
click==8.0.4
cryptography==3.4.8
cycler==0.11.0
Cython==0.29.28
dbus-python==1.2.16
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
distlib==0.3.6
distro==1.4.0
distro-info==0.23+ubuntu1.1
docstring-to-markdown==0.11
entrypoints==0.4
executing==0.8.3
facets-overview==1.0.0
fastjsonschema==2.16.2
filelock==3.9.0
fonttools==4.25.0
idna==3.3
ipykernel==6.15.3
ipython==8.5.0
ipython-genutils==0.2.0
ipywidgets==7.7.2
jedi==0.18.1
Jinja2==2.11.3
jmespath==0.10.0
joblib==1.1.1
jsonschema==4.4.0
jupyter-client==6.1.12
jupyter_core==4.11.2
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.0
kiwisolver==1.3.2
MarkupSafe==2.0.1
matplotlib==3.5.1
matplotlib-inline==0.1.2
mccabe==0.7.0
mistune==0.8.4
mypy-e

Trie, pré-traitement

In [0]:
from pyspark.sql import Window
from pyspark.sql import functions as F


# fonctions de tri 

def replace_text(df):
    text_columns = ["id", "keyword", "location"]

 
    new_text_value = F.coalesce(*[F.when(F.col(c).cast("string").isNotNull() & (F.col(c) != ""), F.col(c)) for c in text_columns])

    df = (df
          .withColumn("text", F.when(F.col("text").isNull(), new_text_value).otherwise(F.col("text")))
          .withColumn("id", F.when((F.col("text") == F.col("id")) & F.col("id").cast("string").isNotNull(), None).otherwise(F.col("id")))
          .withColumn("keyword", F.when((F.col("text") == F.col("keyword")), None).otherwise(F.col("keyword")))
          .withColumn("location", F.when((F.col("text") == F.col("location")), None).otherwise(F.col("location")))
         )
    return df



def replace_target(df):
    
    int_columns = ["keyword", "location", "text"]

    
    new_target_value = F.coalesce(*[F.when((F.col(c) == 0) | (F.col(c) == 1), F.col(c)) for c in int_columns])

    df = (df
          .withColumn("target", F.when(F.col("target").isNull(), new_target_value).otherwise(F.col("target")))
          .withColumn("keyword", F.when((F.col("target") == F.col("keyword")), None).otherwise(F.col("keyword")))
          .withColumn("location", F.when((F.col("target") == F.col("location")), None).otherwise(F.col("location")))
          .withColumn("text", F.when((F.col("target") == F.col("text")), None).otherwise(F.col("text"))))

    return df

def replace_location(df):
    df = df.withColumn("location", F.when(F.col("location").contains("?"), None).otherwise(F.col("location")))
    return df

def drop_null_target(df):
    df = df.filter(F.col("target").isNotNull())
    return df
    
def drop_null_text(df):
    df = df.filter(F.col("text").isNotNull())
    return df



def reset_ids(df):
    
    window_spec = Window.orderBy(F.lit(1))
    df = df.withColumn("id", F.row_number().over(window_spec))
    return df


In [0]:
# on verifie
df2=replace_text(df_train)
null_count = df2.select(F.sum(F.col("target").isNull().cast("int"))).alias("null_count")
null_count.show()
df3=replace_target(df2)
null_count = df3.select(F.sum(F.col("target").isNull().cast("int"))).alias("null_count")
null_count.show()

df3.show(100)

+----------------------------------+
|sum(CAST((target IS NULL) AS INT))|
+----------------------------------+
|                              1211|
+----------------------------------+

+----------------------------------+
|sum(CAST((target IS NULL) AS INT))|
+----------------------------------+
|                               775|
+----------------------------------+

+----+--------+--------------------+--------------------+------+
|  id| keyword|            location|                text|target|
+----+--------+--------------------+--------------------+------+
|   1|    null|                null|Our Deeds are the...|     1|
|   4|    null|                null|Forest fire near ...|     1|
|   5|    null|                null|All residents ask...|     1|
|   6|    null|                null|13,000 people rec...|     1|
|   7|    null|                null|Just got sent thi...|     1|
|   8|    null|                null|#RockyFire Update...|     1|
|  10|    null|                null|#flood 

In [0]:
def tri(df):
    df = replace_text(df)  # remplace le text null, si il existe dans une des autres colonnes
    df = replace_target(df) # remplace le target null si il existe dans une autre colonne
    df = replace_location(df)  #supprime le contenue de location si il y'à au moin un "?"
    df = drop_null_target(df) # supprime les target qui n'ont pas été remplacé car manquant
    df = drop_null_text(df)   # supprime le text devenue nulle , si le target était à l'intérieur
    df = reset_ids(df)  # Réinitialisation des IDs correctement
    return df

In [0]:
data = tri(df_train)
data.show(400)

+---+-------------------+--------------------+--------------------+------+
| id|            keyword|            location|                text|target|
+---+-------------------+--------------------+--------------------+------+
|  1|               null|                null|Our Deeds are the...|     1|
|  2|               null|                null|Forest fire near ...|     1|
|  3|               null|                null|All residents ask...|     1|
|  4|               null|                null|13,000 people rec...|     1|
|  5|               null|                null|Just got sent thi...|     1|
|  6|               null|                null|#RockyFire Update...|     1|
|  7|               null|                null|#flood #disaster ...|     1|
|  8|               null|                null|I'm on top of the...|     1|
|  9|               null|                null|There's an emerge...|     1|
| 10|               null|                null|I'm afraid that t...|     1|
| 11|               null|

ETUDE 1: prise en compte uniquement du text comme variable explicative (location et keyword laisser de coté)

In [0]:
data.count()
data1= data.select("id","text","target")
data1.show()

+---+--------------------+------+
| id|                text|target|
+---+--------------------+------+
|  1|Our Deeds are the...|     1|
|  2|Forest fire near ...|     1|
|  3|All residents ask...|     1|
|  4|13,000 people rec...|     1|
|  5|Just got sent thi...|     1|
|  6|#RockyFire Update...|     1|
|  7|#flood #disaster ...|     1|
|  8|I'm on top of the...|     1|
|  9|There's an emerge...|     1|
| 10|I'm afraid that t...|     1|
| 11|Three people died...|     1|
| 12|Haha South Tampa ...|     1|
| 13|#raining #floodin...|     1|
| 14|#Flood in Bago My...|     1|
| 15|Damage to school ...|     1|
| 16|      What's up man?|     0|
| 17|       I love fruits|     0|
| 18|    Summer is lovely|     0|
| 19|   My car is so fast|     0|
| 20|What a goooooooaa...|     0|
+---+--------------------+------+
only showing top 20 rows



In [0]:
data1 = data1.withColumn("target", F.col("target").cast("int"))
data1 = data1.withColumnRenamed("target", "label")
data1.show()

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Our Deeds are the...|    1|
|  2|Forest fire near ...|    1|
|  3|All residents ask...|    1|
|  4|13,000 people rec...|    1|
|  5|Just got sent thi...|    1|
|  6|#RockyFire Update...|    1|
|  7|#flood #disaster ...|    1|
|  8|I'm on top of the...|    1|
|  9|There's an emerge...|    1|
| 10|I'm afraid that t...|    1|
| 11|Three people died...|    1|
| 12|Haha South Tampa ...|    1|
| 13|#raining #floodin...|    1|
| 14|#Flood in Bago My...|    1|
| 15|Damage to school ...|    1|
| 16|      What's up man?|    0|
| 17|       I love fruits|    0|
| 18|    Summer is lovely|    0|
| 19|   My car is so fast|    0|
| 20|What a goooooooaa...|    0|
+---+--------------------+-----+
only showing top 20 rows



In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

documentAssembler = DocumentAssembler() \
                      .setCleanupMode("inplace") \
                      .setInputCol("text") \
                      .setOutputCol("document")

documents = documentAssembler.transform(data1.limit(5))
documents.show(5, 100)

+---+----------------------------------------------------------------------------------------------------+------+----------------------------------------------------------------------------------------------------+
| id|                                                                                                text|target|                                                                                            document|
+---+----------------------------------------------------------------------------------------------------+------+----------------------------------------------------------------------------------------------------+
|  1|                               Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all|     1|[{document, 0, 68, Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all, {senten...|
|  2|                                                              Forest fire near La Ronge Sask. Canada|     1|                    [{docum

In [0]:
tokenizer = Tokenizer() \
                  .setInputCols(["document"]) \
                  .setOutputCol("token")

tokensEch = tokenizer.fit(documents).transform(documents)
tokensEch.select("token").show(1, 300)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                       token|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{token, 0, 2, Our, {sentence -> 0}, []}, {token, 4, 8, Deeds, {sentence -> 0}, []}, {toke

In [0]:
embeddings = WordEmbeddingsModel.pretrained("glove_6B_300", "xx") \
                                .setInputCols("document", "token") \
                                .setOutputCol("glove_embeddings")

embedsEch = embeddings.transform(tokensEch)
embedsEch.select("glove_embeddings").show(2, 200)

glove_6B_300 download started this may take some time.
Approximate size to download 426.2 MB
[OK!]
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                        glove_embeddings|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{word_embeddings, 0, 2, Our, {isOOV -> false, pieceId -> -1, isWordStart -> true, token -> Our, sentence -> 0}, [-0.025179, 0.07837, -0.17637, -0.12285, -0.13156, 0.26591, -0.60064, 0.46653, 0.042...|
|[{word_embeddings, 0, 5, Forest, {isOOV -> false, pieceId -> -1, isWordStart -> true, to

In [0]:
embeddingsSentence = SentenceEmbeddings() \
                           .setInputCols(["document", "glove_embeddings"]) \
                           .setOutputCol("sentence_embeddings_glove") \
                           .setPoolingStrategy("AVERAGE")


embeddingsFinisher = EmbeddingsFinisher() \
                           .setInputCols("sentence_embeddings_glove") \
                           .setOutputCols("finished_embeddings_glove") \
                           .setOutputAsVector(True) \
                           .setCleanAnnotations(False)

sentencesEch = embeddingsSentence.transform(embedsEch)
embeddingsFinisher.transform(sentencesEch).select("finished_embeddings_glove").show(5, 200)



+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                               finished_embeddings_glove|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[-0.06928376853466034,-0.01271053310483694,0.00862519908696413,-0.25277167558670044,-0.12423738092184067,0.07902661710977554,-0.07260183990001678,0.152671679854393,0.07146238535642624,-1.411920666...|
|[[-0.05899748578667641,0.054601624608039856,-0.1071816235780716,-0.11128462851047516,-0.03310374915599823,-0.08629648387432098,-0.10258549451828003,0.19005662202835083,-0.0949773639440536

In [0]:
from pyspark.ml import Pipeline

pipelineGlove = Pipeline() \
                      .setStages([ \
                         documentAssembler, \
                         tokenizer, \
                         embeddings, \
                         embeddingsSentence, \
                         embeddingsFinisher \
                      ])


resultGlove = pipelineGlove.fit(data1).transform(data1).cache()
resultGlove.select("finished_embeddings_glove").show(5, 200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                               finished_embeddings_glove|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[-0.06928376853466034,-0.01271053310483694,0.00862519908696413,-0.25277167558670044,-0.12423738092184067,0.07902661710977554,-0.07260183990001678,0.152671679854393,0.07146238535642624,-1.411920666...|
|[[-0.05899748578667641,0.054601624608039856,-0.1071816235780716,-0.11128462851047516,-0.03310374915599823,-0.08629648387432098,-0.10258549451828003,0.19005662202835083,-0.0949773639440536

In [0]:
allEmbeddings = resultGlove.selectExpr("finished_embeddings_glove[0]", "label")
partitions = allEmbeddings.randomSplit([0.1, 0.9], seed=1)
apprentissage = partitions[1]
validation = partitions[0]
validation.count()




Out[40]: 757

In [0]:
from pyspark.ml.classification import LinearSVC


linSvcGlove = LinearSVC().setMaxIter(10) \
                         .setFeaturesCol("finished_embeddings_glove[0]") \
                         .setLabelCol("label")


pipelineSvcGlove = Pipeline().setStages([linSvcGlove])

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Construction de la grille de (hyper)paramètres utilisée pour grid search
# Une composante est ajoutée avec .addGrid() pour chaque (hyper)paramètre à explorer
paramGrid = ParamGridBuilder().addGrid(linSvcGlove.regParam, \
                                 [0.02, 0.1, 0.2, 0.5]) \
                              .build()

# Définition de l'instance de CrossValidator : à quel estimateur l'appliquer,
#  avec quels (hyper)paramètres, combien de folds, comment évaluer les résultats
cvSvcGlove = CrossValidator().setEstimator(pipelineSvcGlove) \
                             .setEstimatorParamMaps(paramGrid) \
                             .setNumFolds(5) \
                             .setEvaluator(BinaryClassificationEvaluator())

In [0]:
model = cvSvcGlove.fit(apprentissage)

In [0]:
resultatsAppSvcGlove = cvSvcGloveModel.transform(apprentissage)

# Calculer les prédictions sur les données de test
resultatsSvcGlove = cvSvcGloveModel.transform(validation)

print("AUC sur train : ", cvSvcGloveModel.getEvaluator().evaluate(resultatsAppSvcGlove))

# Calculer AUC sur données de test
print("AUC sur test : ", cvSvcGloveModel.getEvaluator().evaluate(resultatsSvcGlove))


AUC sur train :  0.859730273078728
AUC sur test :  0.8352652358777073
